In [ ]:
# import libs
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.matlib

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from scipy import stats
from scipy.stats import norm
from joblib import Parallel, delayed

import shutil
import glob

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold


import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts


from sklearn import preprocessing, model_selection
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib


path_submissions = '/'

target_name = 'target'
scores_folds = {}

In [ ]:
def transform_csv2pickle(path, usecols, dtype):
    train = pd.read_csv(
        path,
        usecols=usecols,
        dtype=dtypes
    )
    train.to_pickle('train.pkl')


path = '../input/ubiquant-market-prediction/train.csv'

basecols = ['row_id', 'time_id', 'investment_id', 'target']
features = [f'f_{i}' for i in range(300)]

dtypes = {
    'row_id': 'str',
    'time_id': 'uint16',
    'investment_id': 'uint16',
    'target': 'float32',
}
for col in features:
    dtypes[col] = 'float32'

In [ ]:
%%time
train = pd.read_pickle('../input/ump-train-picklefile/train.pkl')

In [ ]:
train = train[train.time_id>1100]
train

In [ ]:
# kfold based on the knn++ algorithm

out_train = train.pivot(index='time_id', columns='investment_id', values='target')

out_train = out_train.fillna(out_train.mean())
out_train.head()

# NN

In [ ]:
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
import numpy as np
from keras import backend as K
from keras.models import load_model
# data separation based on knn ++
nfolds = 5 # number of folds
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds) # number of individuals

# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]


lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)

lineNumber = np.sort(lineNumber)[::-1]

for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))
    
# saves index
for n in range(nfolds):
    
    values.append([lineNumber[n]])    


s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
         # saves the values of index           

        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=0,
    mode='min')

In [ ]:
colNames = list(train)

colNames.remove('time_id')
colNames.remove('target')
colNames.remove('row_id')
colNames.remove('investment_id')

In [ ]:
test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')

In [ ]:
#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
hidden_units = (128,64,32)
embedding_size = 24

cat_data = train['investment_id']

def base_model():
    inv_id_input = keras.Input(shape=(1,), name='investment_id')
    num_input = keras.Input(shape=(len(colNames),), name='num_data')


    #embedding, flatenning and concatenating
    inv_embedded = keras.layers.Embedding(max(cat_data)+1, embedding_size, 
                                           input_length=1, name='investment_embedding')(inv_id_input)
    inv_flattened = keras.layers.Flatten()(inv_embedded)
    out = keras.layers.Concatenate()([inv_flattened, num_input])


    # Add one or more hidden layers
    out = keras.layers.Dense(4096, activation='relu')(out)
    out = keras.layers.Reshape((256, 16))(out)
    out = keras.layers.Conv1D(filters=16, kernel_size=5, strides=1, activation='relu')(out)
    out = keras.layers.MaxPooling1D(pool_size=2)(out)
    out = keras.layers.Flatten()(out)

    #out = keras.layers.Concatenate()([out, num_input])
    out = keras.layers.Dense(16, activation='relu')(out)
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    
    model = keras.Model(
    inputs = [inv_id_input, num_input],
    outputs = out,
    )
    
    return model

In [ ]:
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=42)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train)

features_to_consider.remove('time_id')
features_to_consider.remove('target')
features_to_consider.remove('row_id')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train[features_to_consider] = train[features_to_consider].fillna(train[features_to_consider].mean())
test[features_to_consider] = test[features_to_consider].fillna(test[features_to_consider].mean())

train[pred_name] = 0
test['target'] = 0


for n_count in range(n_folds):
    print('CV {}/{}'.format(counter, n_folds))
    
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    
    X_train = train.loc[train.time_id.isin(indexes), features_to_consider]
    y_train = train.loc[train.time_id.isin(indexes), target_name]
    X_test = train.loc[train.time_id.isin(values[n_count]), features_to_consider]
    y_test = train.loc[train.time_id.isin(values[n_count]), target_name]
    
    
    model = base_model()
    model.summary()
    model.compile(
        keras.optimizers.Adam(learning_rate=0.0001),
        loss='mse'
    )
    
    try:
        features_to_consider.remove('investment_id')
    except:
        pass
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    
    
    cat_data = X_train['investment_id']    
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['investment_id']

    model.fit([cat_data, num_data], 
              target,               
              batch_size=1024,
              epochs=100,
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
             verbose = 1)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    print('score:',pd.DataFrame({"y_train":y_test, "oof_train": preds}).corr()["y_train"]["oof_train"])
    
    tt =scaler.transform(test[features_to_consider].values)
    test[target_name] += model.predict([test['investment_id'], tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    model.save(f'nn_{n_count}.h5')
    counter += 1
    features_to_consider.append('investment_id')
    del model
    gc.collect()

In [ ]:
from keras.models import load_model
test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
features_to_consider = test.drop(['row_id','time_id','investment_id'],axis=1).columns
nn0 = load_model('./nn_0.h5')
nn1 = load_model('./nn_1.h5')
nn2 = load_model('./nn_2.h5')
nn3 = load_model('./nn_3.h5')
nn4 = load_model('./nn_4.h5')


In [ ]:
# inference
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.drop(["row_id"], axis=1)
    tt =scaler.transform(test_df[features_to_consider].values)
    pred_nn0 = nn0.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn1 = nn1.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn2 = nn2.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn3 = nn3.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn4 = nn4.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100

    sample_prediction_df['target'] =  (pred_nn0 + pred_nn1 + pred_nn2 + pred_nn3 + pred_nn4) / 5  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions